This notebook uploads two outputs to Google Cloud Storage buckets.

1) The inputs array converted to a vcf-style hail table.
2) The next upload is a JSON file of the input variants converted to GA4GH GKS VA/VRS annotations using gnomAD data.

It can read from a locally-staged hail table directory, or use one in GCS. The table must have a pre-computed `.info.VRS` field. This is computed using `tgg_methods` `vrs_annotation_batch.py`, which uses the `vrs-python` `vcf_annotation.py`

https://github.com/broadinstitute/tgg_methods/blob/master/vrs/vrs_annotation_batch.py (last checked at `a0002f02fbd5dd25487b261e94081a3daec29c64`)

This validates gnomad_methods functions `gnomad_gks` and `get_gks`, off the branch in this pull request: https://github.com/broadinstitute/gnomad_methods/pull/556

The JSON schema is off the `aw-lb-drafting` branch (at `bf3b5aa`)

The cell that patches the gnomad_gks function can be removed if the python environment this notebook is in is run off the version of code in the gnomad_methods PR 556.

In [ ]:
import hail as hl

In [ ]:
# The clingen-public bucket will only be the week of 2023-09-18
# The clingen-public-requesterpays bucket will remain available but the client will need to 
# be authenticated with a Google Cloud account with billing enabled to pay network transfer fees

# configuration for data outputs
bucket = "clingen-public-requesterpays"
bucket = "clingen-public"

# Writes inputs array as a hail table to this destination, if not None.
# This can be useful for other testing, using this hail table as input without reconstructing it
# inputs_ht_destination_url = f"gs://{bucket}/gnomad-gks-qc/gnomad-filtered-inputs.ht"
inputs_ht_destination_url = None

# Copies output annotations as newline delimited json to this url, if not None
outputs_destination_file = f"gs://{bucket}/gnomad-gks-qc/outputs.ndjson"


# ht_url can be a gs:// path, or a file:// local path

# Publicly readable, but doesn't have all gnomad variants in it
# ht_url = f"gs://{bucket}/gnomad-gks-downsampled-100k.ht"

# Can refer to a local hail table directory
ht_url = "../downsample_to_100k_full_release.ht"

# gnomad-vrs-io-finals is a private bucket, needs a service account with read access 
# ht_url = "gs://gnomad-vrs-io-finals/ht-outputs/release_0426_dmtest_v3.1.2-Full-ht-release-output-updated-schema-050523.ht"


print(ht_url)
ht = hl.read_table(ht_url)

In [ ]:
import gnomad
import gnomad.utils.annotations
import gnomad.resources.grch38.gnomad

# reload (re-running this cell will reload modifications to these modules on disk)
import importlib
importlib.reload(gnomad.utils.annotations)
importlib.reload(gnomad.resources.grch38.gnomad)

from gnomad.utils.annotations import add_gks_va, add_gks_vrs
from gnomad.resources.grch38.gnomad import gnomad_gks

In [ ]:
import subprocess
import shutil
va_spec_clone = "gnomad-gks-v1_va-spec"
va_spec_branch = "gk-pilot"
shutil.rmtree(va_spec_clone, ignore_errors=True)
p = subprocess.run(["git", "clone", "https://github.com/ga4gh/va-spec", "gnomad-gks-v1_va-spec"],
                   check=True)
p = subprocess.run(["bash", "-c",
                    f"cd {va_spec_clone} && git checkout {va_spec_branch}"],
                   check=True)

In [ ]:
import json
with open(f"{va_spec_clone}/schema/cohortAlleleFreq.json") as f:
    schema = json.load(f)

In [ ]:
# GnomAD 3.1.2 
# GRCh38 expressions

inputs = [
    {
        "clinvar": "422227",
        "gnomad": "16-68737366-C-T"
    },
    {
        "clinvar": "619",
        "gnomad": "12-102843683-C-T"
    },
    {
       "clinvar": "808527",
       "gnomad": "19-38442434-C-T"
    },
    {
        "clinvar": "695823",
        "gnomad": "1-11130641-G-A"
    },
    # Not in gnomAD 3.1.2
#     {
#         "clinvar": "251162",
#         "gnomad": "19-11105249-C-T"
#     },
    {
        "clinvar": "143344",
        "gnomad": "X-154030690-C-T"
    },
    {
        "clinvar": "137390",
        "gnomad": "14-28767768-C-T"
    },
    {
        "clinvar": "129997",
        "gnomad": "15-89317458-C-G"
    },
    {
        "clinvar": "89198",
        "gnomad": "2-47799491-C-G"
    },
    {
        "clinvar": "43032",
        "gnomad": "14-23416241-G-A"
    },
    {
        "clinvar": "2356",
        "gnomad": "1-216247118-C-A"
    },
    {
        "clinvar": "3038",
        "gnomad": "11-108248927-T-G"
    },
    {
        "clinvar": "256155",
        "gnomad": "7-107701083-T-A"
    },
    # example variant from va-spec simple_result_example.yaml
    {
        "gnomad": "1-55051215-G-GA"
    }
]


In [ ]:
# This cell constructs an indexed hail table from the input alleles
# so the full gnomad dataset can be rapidly filtered by left joining it onto this.
# Using a .filter() to check membership in the input set is slow because it needs
# to do a table scan.
# Too slow:
# ht_with_coords.filter(input_gnomad_expressions.contains(ht_with_coords.genomic_coordinates))

import pandas

input_gnomad_expressions = hl.literal([x["gnomad"] for x in inputs])
input_terms = [x["gnomad"].split("-") for x in inputs]

df = pandas.DataFrame(
    {
        "contig": [str("chr" + i[0]) for i in input_terms],
        "position": [int(i[1]) for i in input_terms],
        "ref": [i[2] for i in input_terms],
        "alt": [i[3] for i in input_terms]
    }
)
input_ht = hl.Table.from_pandas(df)
input_ht = (input_ht
    .annotate(
        locus=hl.locus(input_ht.contig, input_ht.position, reference_genome="GRCh38"),
        alleles=hl.array([input_ht.ref, input_ht.alt]))
    .drop("contig", "position", "ref", "alt")
    .key_by("locus", "alleles"))

input_ht.show()

In [ ]:
ht_with_coords = ht.annotate(
    genomic_coordinates = hl.format("%s-%s-%s-%s",
        ht.locus.contig[3:], # Remove 'chr'
        hl.str(ht.locus.position),
        ht.alleles[0],
        ht.alleles[1]
    )
)
ht_with_coords.show()

ht_filtered = ht_with_coords

# Indexed filter using join on the indexed vcf table constructed from inputs
# To enable filtering to the set of input variants, uncomment everything below

# ht_filtered = input_ht.join(ht_with_coords)
# ht_filtered.genomic_coordinates.show()

# # Write the filtered gnomad table to storage
# if inputs_ht_destination_url:
#     ht_filtered.write(inputs_ht_destination_url, overwrite=True)

In [ ]:
# Parameters for gnomad_gks/get_gks
ancestry_group_short_names = gnomad.resources.grch38.gnomad.POPS["v3"]
ancestry_groups_full_name_map = gnomad.sample_qc.ancestry.POP_NAMES
gnomad_version_label = "3.1.4"

In [ ]:
vrs_variants = []
records = ht_filtered.select(
    ht_filtered.freq,
    ht_filtered.popmax,
    ht_filtered.info,
    ht_filtered.genomic_coordinates,
    vrs=ht_filtered.info.vrs
)
records = records.take(5)

variant_strs = [r.genomic_coordinates for r in records]
loci = [
    hl.locus(contig=str("chr" + v0), pos=int(v1), reference_genome="GRCh38")
    for [v0, v1, *_] in 
    [v.split("-") for v in variant_strs]
]

print(f"gnomAD allele strings: {variant_strs}")
for record in records:
    print("calling add_gks_vrs on: " + record.genomic_coordinates)
    vrs_variant = add_gks_vrs(
        input_vrs=record.info.vrs,
        input_locus=record.locus
    )
    print(json.dumps(vrs_variant, indent=2))
    vrs_variants.append(vrs_variant)


In [ ]:
# This cell takes one locus present in the input table, performs VRS and VA annotation and prints it.

# For performance on larger datasets, the interval should be much larger, at least a few thousand.

ivl_0 =hl.locus_interval(loci[0].contig, loci[0].position, loci[0].position+1, reference_genome="GRCh38")
ht_locus_0 = (hl.filter_intervals(ht_filtered, [ivl_0]))

import time
t0 = time.time()
gks_annotation = gnomad_gks(
    locus_interval=ivl_0,
    custom_ht=ht_filtered,
    version="3.1.4",
    data_type="genomes",
    by_ancestry_group=False,
    by_sex=False,
    vrs_only=False,
    skip_checkpoint=True,
    skip_coverage=True
)
t1 = time.time()
print(f"{t1-t0} seconds")


gks_annotation = gnomad_gks(
    locus_interval=ivl_0,
    custom_ht=ht_filtered,
    version="3.1.4",
    data_type="genomes",
    by_ancestry_group=False,
    by_sex=False,
    vrs_only=False,
    skip_checkpoint=True,
    skip_coverage=False
)
t1 = time.time()
print(f"{t1-t0} seconds")


print(json.dumps(gks_annotation[0], indent=2))

In [ ]:
# This cell takes one locus present in the input table, performs VRS and VA annotation and prints it.

# For performance on larger datasets, the interval should be much larger, at least a few thousand.
print(hl.eval(loci[0].contig))
ivl_0 =hl.locus_interval(loci[0].contig, loci[0].position, loci[0].position+1, reference_genome="GRCh38")
ht_locus_0 = (hl.filter_intervals(ht_filtered, [ivl_0]))


larger_interval = hl.locus_interval(contig="chr1",
                       start=1,
                       end=int(1e7),
                       reference_genome="GRCh38")

import time
t0 = time.time()
gks_annotations = gnomad_gks(
    locus_interval=larger_interval,
    custom_ht=ht_filtered,
    version="3.1.4",
    data_type="genomes",
    by_ancestry_group=False,
    by_sex=False,
    vrs_only=False,
    skip_checkpoint=True,
    skip_coverage=False
)
t1 = time.time()
l = len(gks_annotations)
td = t1 - t0
print(f"Annotated {l} records in {td} seconds ({td/l:.2f} sec/rec)")

print(json.dumps(gks_annotation[0], indent=2))